# Import required modules

In [ ]:
import pandas as pd
import numpy as np
import scipy

# Input the language

In [ ]:
lang = input('Choose language: 1 for tamil, 2 for malayalam, 3 for kannada: ')

class_list = ['Not_offensive',
 'Offensive_Targeted_Insult_Group',
 'Offensive_Targeted_Insult_Individual',
 'Offensive_Targeted_Insult_Other',
 'Offensive_Untargetede']


if lang == '1': 
    lang = 'Tamil'
    class_list.append('not-Tamil')
elif lang == '2': 
    lang = 'malayalam'
    class_list.append('not-malayalam')
    class_list.remove('Offensive_Targeted_Insult_Other')
elif lang == '3': 
    lang = 'Kannada'
    class_list.append('not-Kannada')

# Enter paths of input files

In [ ]:
mbert_output_file = input("Enter the path(absolute or relative) for the .xlsx file obtained from mBERT: ")
ulmfit_output_file = input("Enter the path(absolute or relative) the .xlsx file obtained from ULMFiT: ")

# Make their dataframes

In [ ]:
mbert_df = pd.read_excel(mbert_output_file, engine='openpyxl')
ulmfit_df = pd.read_excel(ulmfit_output_file, engine='openpyxl')

In [ ]:
mbert_df = mbert_df.drop(columns='Unnamed: 0')
mbert_df = mbert_df.rename(columns={'Input': 'query', 'Correct Label': 'actual_label'})

In [ ]:
mbert_df = mbert_df.drop(columns=['query'])

# Concatenate the dataframes

In [ ]:
main_df = pd.concat([mbert_df, ulmfit_df], axis=1)
main_df

In [ ]:
req_cols = class_list

# The soft voting

In [ ]:
def max_cols(row):
    maxm = []
    for col in req_cols:
        maxm.append((col, row[col] + row['Score_'+col]))
    row['max_both'] = max(maxm, key=lambda x: x[1])[0]

    return pd.Series(row)

In [ ]:
final_df = main_df.apply(lambda x: max_cols(x), axis=1)
final_df

In [ ]:
final_df.head(10)

# Write results to a file

In [ ]:
final_df[['query', 'max_both']].to_excel(f'final_output_{lang}.xlsx', header=None, index=False, encoding='utf-8')